### SINCE WE ARE COMPARING MODELS ON MCI, AD, DATA, WE DO NOT NEED TO ADD IN K-FOLD CROSS FOLDING. Since we're not comparing models reconstruction/clustering on CN, but for MCI and AD.

In [ ]:
# IMPORT EVERYTHING
from Architectures.AE_ADJ import AE  # for some reason importing this makes it not work? (reconstructs everything around the brain somehow)

from Data.load import load, load_middle_slices
from Visualizations.latent_space_projections import pPCA, pTSNE, pUMAP, pISOMAP, pENCODED, plot_multiple_datasets
from Visualizations.plots import plot_middle_slices_in_range, plot_models_training_time, compare_models_loss_history, plot_images, compare_models_reconstruction
from Metrics.metrics_tf import MSE_loss, NMSE_loss, NRMSE_loss, SSIM_loss
from Metrics.metrics import NMSE, SSIM, NRMSE, MSE

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import layers, Model, activations, regularizers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from skimage.metrics import structural_similarity as ssim

import umap
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd
import seaborn as sns
import glob
import tensorflow as tf
import time
import math
import nibabel as nib
import json
from tqdm import tqdm

from sklearn.model_selection import GroupKFold

c:\Users\kiran\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from glob import glob
import numpy as np
import nibabel as nib
from tqdm import tqdm
from scipy.ndimage import zoom
import os
from tensorflow.keras import utils

def load_k_fold(
        dataset_name:str = "Pre-processed",
        crop:bool = True,
        normalize:bool = True,
        #subvoxel_rolling_augmentation:bool = True,
        target_size:tuple|None = (80,96,80),
        train_test_split:float = 0.9,
        take:int=-1,
        subdirs=[]
        ):
    """
    :params:
    - dataset_name: Path relative to this script to the parent folder of the dataset.
    - crop: If True: images are cut in all axes s.t. only the non-zero voxels are included.
    - normalize: If True: magnitude of voxels are normalized between 0 and 1.
    - target_size: Downsampling target size. Set to None to resample to lowest cropped size.
    - train_test_split: The ratio of images used in the train set. Set to 1 to include all loaded images in a single dataset.
    - take: How many images to load. Set to -1 to load all the images in the dataset.
    """
    #- subvoxel_rolling_augmentation: If True: rolls the images around some times, producing a richer dataset.
    if len(subdirs) > 0:
        x_train = []
        x_test = []
        y_train = []
        y_test = []
        for i, group in enumerate(subdirs):
            data = load(
                f"{dataset_name}/{group}",
                crop,
                normalize,
                target_size,
                train_test_split,
                take
                )
            if len(data) == 2:
                x_train.extend(data[0])
                x_test.extend(data[1])
                y_train.extend([i]*len(data[0]))
                y_test.extend([i]*len(data[1]))
            else:
                x_train.extend(data)
                y_train.extend([i]*len(data))

        y_train = utils.to_categorical(y_train)
        y_test = utils.to_categorical(y_test)
        x_train = np.stack(x_train)
        x_test = np.stack(x_test)
        return x_train, y_train, x_test, y_test
    
    script_dir = os.path.dirname(os.path.abspath(__file__))  # Get the directory of this script
    dataset_path = os.path.join(script_dir, dataset_name)  # Create the absolute path
    files = glob(os.path.join(dataset_path, "*.nii.gz"))  # Use absolute path for glob
    if take >= 0:
        files = files[:take]

    images = []

    def handle_image(img):
        if crop:
            brain_mask = img > 0.01
            bounds = np.where(brain_mask)
            x_min, x_max, y_min, y_max, z_min, z_max = np.min(bounds[0]), np.max(bounds[0]), np.min(bounds[1]), np.max(bounds[1]), np.min(bounds[2]), np.max(bounds[2])
            img = img[x_min-2:x_max+3, y_min-2:y_max+3, z_min-2:z_max+3]

        '''
        if subvoxel_rolling_augmentation:
            # Since we zoom the image, we can nudge the picture in various directions to
            # achieve new sub-pixel level information in the output :)
            for roll in range(2):
                for axis in range(3):
                    loaded_images.append(np.roll(img, roll*2, axis))
        '''
                    
        if normalize:
            q2m = .785700/.475665
            fac = np.min([q2m / np.quantile(img,0.98), 1. / np.max(img)])
            img *= fac
            
        if target_size != None:
            zoom_factors = [t / b for t, b in zip(target_size, img.shape)]
            img = zoom(img, zoom_factors, order=1)  # Linear interpolation


        return [img]
    
    desc = f"Loading {dataset_name.split('/')[-1]}"
    for file in tqdm(files, desc):
        img = nib.load(file).get_fdata()
        images.extend(handle_image(img))

    images = np.stack(images)
    if train_test_split>=1 or train_test_split<=0: return images

    idx_split = int(len(images) * train_test_split)
    train, test = images[:idx_split], images[idx_split:]

    return train, test

groups = {}
for group in tqdm(["CN","MCI","AD"],"Opening faster"):
    groups[group]=np.load(open(f"Data/D2-{group}-A.npy","br"))

Opening faster: 100%|██████████| 3/3 [00:00<00:00, 32.13it/s]


In [57]:
from glob import glob
import numpy as np
import nibabel as nib
from tqdm import tqdm
from scipy.ndimage import zoom
import os
from tensorflow.keras import utils


take = 100
normalize = True
dataset_name = "Pre-processed"
crop = True
normalize=  True
#subvoxel_rolling_augmentation:bool = True,
target_size= (80,96,80)
train_test_split= 0.9
take: -1
#script_dir = os.path.dirname(os.path.abspath(__file__))  # Get the directory of this script
#dataset_path = os.path.join(script_dir, dataset_name)  # Create the absolute path

dataset_path = r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\Data\Pre-processed-CN"
files = glob(os.path.join(dataset_path, "*.nii.gz"))
if take >= 0:
    files = files[:take]

image_ids = []
for file in files:
    #C:\\blabla\106_ADNI_116_S_0487_blabla.nii.gz -> 116_S_0487_MR_blabla.nii.gz -> 116_S_0487 (pID)
    #C:\1_sub-ADNI002S0413_ses-M132_T1w2_norm.nii.gz -> C:\1_sub_ADNI002S0413_ses-M132_T1w2_norm.nii.gz -> 002S0413_ses-M132_T1w2_norm.nii.gz -> 002S0413
    if "_sub-ADNI" in file:
        file = file.split("_sub-ADNI")[-1]
        file = file.split("_ses")[0] # 002S0413
        file_id = "_S_".join(file.split("S"))
    elif "_ADNI_" in file:
        file = file.split("_ADNI")[-1]
        file_id = file.split("_MR")[0]
    else:
        print(f"Problems extracting id from {file}")
        file_id = 0
    image_ids.append(file_id)

images = []

def handle_image(img):
    if crop:
        brain_mask = img > 0.01
        bounds = np.where(brain_mask)
        x_min, x_max, y_min, y_max, z_min, z_max = np.min(bounds[0]), np.max(bounds[0]), np.min(bounds[1]), np.max(bounds[1]), np.min(bounds[2]), np.max(bounds[2])
        img = img[x_min-2:x_max+3, y_min-2:y_max+3, z_min-2:z_max+3]

    '''
    if subvoxel_rolling_augmentation:
        # Since we zoom the image, we can nudge the picture in various directions to
        # achieve new sub-pixel level information in the output :)
        for roll in range(2):
            for axis in range(3):
                loaded_images.append(np.roll(img, roll*2, axis))
    '''
                
    if normalize:
        q2m = .785700/.475665
        fac = np.min([q2m / np.quantile(img,0.98), 1. / np.max(img)])
        img *= fac
        
    if target_size != None:
        zoom_factors = [t / b for t, b in zip(target_size, img.shape)]
        img = zoom(img, zoom_factors, order=1)  # Linear interpolation


    return [img]

desc = f"Loading {dataset_name.split('/')[-1]}"
for file in tqdm(files, desc):
    img = nib.load(file).get_fdata()
    images.extend(handle_image(img))

images = np.stack(images)
if train_test_split>=1 or train_test_split<=0: 
    pass
    #return images

#return train, test
gkf = GroupKFold(n_splits=5)
train_test_folds = []
for train_idx, test_idx in gkf.split(images, y=np.zeros(len(images)), groups=image_ids):
    X_train, X_test = images[train_idx], images[test_idx]
    train_test_folds.append((X_train, X_test))
    #print(len(X_train), len(X_test),"\n", train_idx, test_idx)

Loading Pre-processed: 100%|██████████| 100/100 [00:27<00:00,  3.62it/s]


In [ ]:

take = 10
#script_dir = os.path.dirname(os.path.abspath(__file__))  # Get the directory of this script
#dataset_path = os.path.join(script_dir, dataset_name)  # Create the absolute path
dataset_path = r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\Data\Pre-processed-AD"
files = glob(os.path.join(dataset_path, "*.nii.gz"))  # Use absolute path for glob
if take >= 0:
    files = files[:take]

images = []

def handle_image(img):
    if crop:
        brain_mask = img > 0.01
        bounds = np.where(brain_mask)
        x_min, x_max, y_min, y_max, z_min, z_max = np.min(bounds[0]), np.max(bounds[0]), np.min(bounds[1]), np.max(bounds[1]), np.min(bounds[2]), np.max(bounds[2])
        img = img[x_min-2:x_max+3, y_min-2:y_max+3, z_min-2:z_max+3]

    '''
    if subvoxel_rolling_augmentation:
        # Since we zoom the image, we can nudge the picture in various directions to
        # achieve new sub-pixel level information in the output :)
        for roll in range(2):
            for axis in range(3):
                loaded_images.append(np.roll(img, roll*2, axis))
    '''
                
    if normalize:
        q2m = .785700/.475665
        fac = np.min([q2m / np.quantile(img,0.98), 1. / np.max(img)])
        img *= fac
        
    if target_size != None:
        zoom_factors = [t / b for t, b in zip(target_size, img.shape)]
        img = zoom(img, zoom_factors, order=1)  # Linear interpolation


    return [img]

desc = f"Loading {dataset_name.split('/')[-1]}"
for file in tqdm(files, desc):
    img = nib.load(file).get_fdata()
    images.extend(handle_image(img))

images = np.stack(images)
if train_test_split>=1 or train_test_split<=0: return images

idx_split = int(len(images) * train_test_split)
train, test = images[:idx_split], images[idx_split:]

return train, test

In [ ]:
H_train3D, H_test3D = load(dataset_name="Pre-processed", train_test_split=0.8, take=take)


In [ ]:
# TODO: load all images in a list, also have another list for the patient ids, then 
# 1_ADNI_002_S_1018_MR_MPR____N3__Scaled_Br_20070217032215330_S24312_I408282_norm.nii.gz
# 002_S_1018


In [ ]:

# X = array of images
# y = optional labels #(you might not have these if it's unsupervised)
# groups = patient IDs corresponding to each image

gkf = GroupKFold(n_splits=5)
for train_idx, test_idx in gkf.split(X, y, groups=patient_ids):
    X_train, X_test = X[train_idx], X[test_idx]

# gkf = GroupKFold(n_splits=5)
# for train_idx, test_idx in gkf.split(X, y=np.zeros(len(X)), groups=patient_ids):
#     X_train, X_test = X[train_idx], X[test_idx]

import re
def extract_patient_id(filename):
    match = re.search(r'\d{3}_S_\d{4}', filename)
    return match.group(0) if match else None
filename = "1_ADNI_002_S_1018_MR_MPR____N3__Scaled_Br_20070217032215330_S24312_I408282_norm.nii.gz"
patient_id = extract_patient_id(filename)
print(patient_id)  # ➜ 002_S_1018

